In [1]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json

from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.text import MIMEText
from email.mime.base import MIMEBase

import requests
import smtplib
import pickle

import datetime
import os

In [2]:
KAKAO_TOKEN_SAVED_FILE = "kakao_msg/kakao_token.json"
NAVER_USER_DATA_SAVED_FILE = "naver/account"

def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [3]:
def send_email(smtp_info, naver_user_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info["smtp_port"]) as server:
        server.starttls()
        
        server.login(naver_user_info["smtp_user_id"], naver_user_info["smtp_user_pw"])
        
        res = server.sendmail(msg["from"], msg["to"], msg.as_string())
        
        if not res:
            print("이메일 전송 성공")
        else:
            print(res)
            
def make_multipart(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        if key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
                
        msg.add_header('Content-Disposition', 'attachment', filename = os.path.basename(value['filename']))
        
        multi.attach(msg)
        
    return multi

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/order-test', methods = ['GET'])
def order_test(): #카톡보내는거 이메일보내는거 다 분리해야 좋음
    tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

    f = open(NAVER_USER_DATA_SAVED_FILE, 'rb')
    naver_user_info = pickle.load(f)
    f.close()
    
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    
    headers = {
        "Authorization": "Bearer " + tokens['access_token']
    }
    
    msg = "주문이 완료되었습니다!\n구매자: params로 받아온 Id, 결제금액: Params로 받아온 금액"
    
    data = {
        "template_object": json.dumps({
            "object_type": "text",
            "text": msg,
            "link": {
                "web_url": "https://www.naver.com"
            }
        })
    }
    
    response = requests.post(url, headers=headers, data=data)

    if response.status_code != 200:
        print("에러!")
        
    print("카카오톡 메시지 전송 완료")
    
    smtp_info = dict({
        "smtp_server": "smtp.naver.com",
        "smtp_port": 587
    })
    
    msg_dict = {
        'image': {
            'maintype': 'image',
            'subtype': 'jpg'
        }
    }
    
    title = "xx물품의 결제가 완료되었습니다!" #{0}.format 어쩌구로 처리
    content = "yy에서 xx를 구매하였습니다!\n앞으로도 저희 yy를 많이 이용해주세요"
    sender = naver_user_info['smtp_user_id']
    receiver = naver_user_info['smtp_user_id']
    
    msg = MIMEText(_text = content, _charset = 'utf-8')
    msg_dict['image']['filename'] = 'image/test_1.jpg'
    
    multi = make_multipart(msg_dict)
    
    multi['Subject'] = title
    multi['From'] = sender
    multi['To'] = receiver
    multi.attach(msg)
    
    send_email(smtp_info, naver_user_info, multi)
    
    return jsonify("카톡 메시지 및 이메일 전송 완료")

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


카카오톡 메시지 전송 완료


127.0.0.1 - - [25/May/2022 16:23:25] "GET /order-test HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 16:23:25] "GET /images/icons/gear.png HTTP/1.1" 404 -


이메일 전송 성공
